In [7]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import traceback
from tqdm import tqdm
import time
import json

In [8]:
corp_info = pd.read_csv(r'corporation_information_2016.csv')

#### 보고서 종류별로 저장
- dictionary(key : 종목명_종목코드, value : dictionary) 내의 dictionary(key : 보고서종류, value : dataframe)로 저장
        <주요 보고서 코드>
        http://dart.fss.or.kr/dsap001/guide.do
        A001 : 사업보고서 | A002 : 반기보고서 | A003 : 분기보고서
        B001 : 주요사항보고서 | C001 : 증권신고(지분증권) | C002 : 증권신고(채무증권)
        F001 : 감사보고서 | F002 : 연결감사보고서 | F003 : 결합감사보고서
        

In [9]:
bsn_tp= ['A001','A002','A003','A004','A005',
         'B001','B002','B003',
         'C001','C002','C003','C004','C005','C006','C007','C008','C009','C010','C011',
         'D001','D002','D003','D004',
         'E001','E002','E003','E004','E005','E006','E007','E008','E009',
         'F001','F002','F003','F004',
         'G001','G002','G003',
         'H001','H002','H003','H004','H005','H006',
         'I001','I002','I003','I004','I005','I006',
         'G001','G002','G003','G004','G005','G006']

In [11]:
def constructDictionary():
    storage = {}  # 종목명_종목코드를 key로 사용
    for index, row in tqdm(corp_info.iterrows()):
        storage[row.종목명 + row.종목코드] = {}
    return storage

In [12]:
def getDocumentInfo(crp, doctp): # crp는 corp_keys에서, doctp는 bsn_tp에서 가져온다.
    global doc_dict
    
    crp_cd = crp.split('_')[1]
    auth_key = 'e7064f9e9e2e3bff1fa800140364719a0a33d9e8'
    request_url = 'http://dart.fss.or.kr/api/search.xml?auth=' + auth_key + '&crp_cd=' + crp_cd + '&start_dt=19990101&bsn_tp=' + doctp + '&page_no=1&page_set=100'
    
    r = requests.post(request_url)
    soup = BeautifulSoup(r.text,'lxml')
    #보고서명
    rpt_nm = soup.find_all('rpt_nm')
    rpt_nm = [element.text for element in rpt_nm]
    #접수번호
    rcp_no = soup.find_all('rcp_no')
    rcp_no = [element.text for element in rcp_no]
    #접수일자
    rcp_dt = soup.find_all('rcp_dt')
    rcp_dt = [int(element.text) for element in rcp_dt]
    
    df = pd.DataFrame()
    df['보고서명'] = rpt_nm
    df['접수번호'] = rcp_no
    df['접수일자'] = rcp_dt
    
    doc_dict[crp][doctp] = df
    
    return

In [13]:
doc_dict = constructDictionary()
corp_keys = list(doc_dict.keys())
corp_keys.sort()

2247it [00:00, 8169.51it/s]


In [14]:
for key in tqdm(corp_keys):
    for tp in ['A001','A002','A003','A004','A005']:
        getDocumentInfo(key, tp)
        time.sleep(np.random.randint(1,1000)/500)
    time.sleep(np.random.randint(1,1500)/500)

  1%|          | 26/2247 [03:00<4:27:12,  7.22s/it]

KeyboardInterrupt: 

### Dictionary를 Json파일로 저장

In [8]:
# http://stackoverflow.com/questions/33061302/dictionary-of-panda-dataframe-to-json
# dataframe이 Key로 들어가있는 경우는 일반적인 경우와 다르게 dump해야 함.
class JSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if hasattr(obj, 'to_json'):
            return obj.to_json(orient='records')
        return json.JSONEncoder.default(self, obj)
    
with open('doc_info.json', 'w') as f:
     json.dump(doc_dict, f, cls=JSONEncoder)